In [2]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.linalg import eigvals
import random

In [14]:
def matrix_power(matrix, power):
    """
    Calculates the power of a matrix.

    Parameters:
    matrix (numpy.ndarray): The matrix to calculate the power of.
    power (int): The power to raise the matrix to.

    Returns:
    numpy.ndarray: The resulting matrix.
    """
    return np.linalg.matrix_power(matrix, power)

def matrix_vector_multiplication(matrix, vector):
    """
    Calculates the multiplication of a matrix and a vector.

    Parameters:
    matrix (numpy.ndarray): The matrix to multiply.
    vector (numpy.ndarray): The vector to multiply.

    Returns:
    numpy.ndarray: The resulting vector.
    """
    return np.dot(matrix, vector)

def assign_random_values():
    # between -1, 1
    a = random.uniform(-1, 1)
    b = random.uniform(-1, 1)
    c = random.uniform(-1, 1)
    d = random.uniform(-1, 1)
    e = random.uniform(-1, 1)
    f = random.uniform(-1, 1)
    g = random.uniform(-1, 1)
    h = random.uniform(-1, 1)
    i = random.uniform(-1, 1)
    j = random.uniform(-1, 1)
    k = random.uniform(-1, 1)
    l = random.uniform(-1, 1)
    return a, b, c, d, e, f, g, h, i, j, k, l

def print_multiplication_until_n(G, q, n):
    """
    Prints the multiplication of matrix and vector for k = 1 to n.

    Parameters:
    G (numpy.ndarray): The matrix to multiply.
    q (numpy.ndarray): The vector to multiply.
    n (int): The number of times to multiply the matrix and vector.

    Returns:
    None
    """
    result = np.eye(4)
    for i in range(1, n+1):
        result += (-1)**i * matrix_power(G, i)
        output_k = matrix_vector_multiplication(result, q)
        output_k = np.maximum(output_k, 0)
        print("Multiplication of matrix and vector for k = {}: {}".format(i, output_k))

In [27]:
# our matrix
a, b, c, d, e, f, g, h, i, j, k, l = assign_random_values()
# a = 0
# b = 0.5
# c = 0.5

G = np.array([[0, a, b, c],[d, 0, e, f],[g, h, 0, i],[j, k, l, 0]])
# our vector
# choose a random vector
# q = np.random.rand(4)
q = np.array([1, 1, 1, 1])

print("Matrix G: \n" + str(G))
print("Vector q: " + str(q))
print("\n\n")

Matrix G: 
[[ 0.          0.57018887  0.02938609  0.16039534]
 [ 0.55104383  0.          0.51476827 -0.50460062]
 [-0.84240268  0.02038466  0.         -0.0499997 ]
 [-0.12680347 -0.41446123  0.93103257  0.        ]]
Vector q: [1 1 1 1]





In [28]:
print("Infinity norm of G: " + str(np.linalg.norm(G, ord=np.inf)))
print("Norm-2 of G: " + str(np.linalg.norm(G)))
print("Norm-1 of G: " + str(np.linalg.norm(G, ord=1)))
# print sorted eigenvalues of G
print("Eigenvalues of G: " + str(sorted(eigvals(G))))

Infinity norm of G: 1.570412720429803
Norm-2 of G: 1.7152841371848722
Norm-1 of G: 1.52024998085195
Eigenvalues of G: [(-1.0315386562111777+0j), (0.16807511241867706-0.5999024069921508j), (0.16807511241867706+0.5999024069921508j), (0.6953884313738229+0j)]


In [29]:
print_multiplication_until_n(G, q, 40)

Multiplication of matrix and vector for k = 1: [0.2400297  0.43878852 1.87201772 0.61023213]
Multiplication of matrix and vector for k = 2: [0.59691801 0.2120013  1.22376853 0.        ]
Multiplication of matrix and vector for k = 3: [0.92826515 0.         1.47199332 0.02418923]
Multiplication of matrix and vector for k = 4: [1.08208729 0.         1.78780234 0.        ]
Multiplication of matrix and vector for k = 5: [1.14963586 0.         1.89945826 0.        ]
Multiplication of matrix and vector for k = 6: [1.44014467 0.         1.95086179 0.        ]
Multiplication of matrix and vector for k = 7: [1.61942562 0.         2.18669785 0.        ]
Multiplication of matrix and vector for k = 8: [1.81581153 0.         2.33884479 0.        ]
Multiplication of matrix and vector for k = 9: [2.02184588 0.         2.49331606 0.        ]
Multiplication of matrix and vector for k = 10: [2.2561872  0.         2.66254086 0.        ]
Multiplication of matrix and vector for k = 11: [2.47018359 0.       

In [33]:
# I+G inverse
C = np.linalg.inv(np.eye(4) + G)
np.matmul(q, C)

array([10.01861859, -6.27372001,  7.11630243, -3.41684976])

In [ ]:
"""
f(x) = c_0 + c_1 * x + c_2 * x^2 + ...

If the series converges for |x|<r, 
then the corresponding matrix series converges for matrices A such that ||A||<r 
for some matrix norm that satisfies ||AB||<||A|| ||B||
"""